### Fast conversion from interleaved (int16,int16) to complex64

In [1]:
import cupy as cp
import numpy as np
from cupyx.profiler import benchmark
import numba.cuda

N = int(15.36e6 * 0.1)

#  to hold larger casted float32 dtypes, create an array with double size for each i and q sample
rx_buff = numba.cuda.mapped_array(
    (4 * N,),
    dtype='int16',
    strides=None,
    order='C',
    stream=0,
    portable=False,
    wc=False,
)

rx_buff[::2] = np.random.randint(0, 1000, size=2 * N)


def test0():
    # the most obvious (?) way to do it
    a = cp.array(rx_buff[::2]).astype('float32')
    return a[::2] + np.complex64(1j) * a[1::2]


def test1():
    # use .view to avoid copying and summing. this _may_ have less memory overhead
    return cp.array(rx_buff[::2], copy=False).astype('float32').view('complex64')


def test2():
    # store the cast array in the extra samples of rx_buff
    rx_complex_buff = cp.array(rx_buff, copy=False).view('float32')
    cp.copyto(rx_complex_buff, cp.array(rx_buff)[::2], casting='unsafe')
    return rx_complex_buff.view('complex64')

In [2]:
benchmark(test0, n_repeat=100, n_warmup=100)

test0               :    CPU: 34650.170 us   +/- 86.054 (min: 34512.350 / max: 34983.198) us     GPU-0: 36306.107 us   +/- 77.978 (min: 36180.672 / max: 36633.984) us

In [3]:
benchmark(test1, n_repeat=100, n_warmup=100)

test1               :    CPU: 34352.541 us   +/- 498.704 (min: 34059.390 / max: 37864.254) us     GPU-0: 35069.014 us   +/- 495.717 (min: 34768.448 / max: 38565.567) us

In [4]:
benchmark(test2, n_repeat=100, n_warmup=100)

test2               :    CPU:   455.759 us   +/- 82.699 (min:   397.440 / max:   905.824) us     GPU-0:  2497.480 us   +/- 123.409 (min:  2148.800 / max:  2823.840) us

In [5]:
a0 = test0()
a1 = test1()
a2 = test2()
print('max error: ', cp.abs(a0 - a1).max(), cp.abs(a0 - a2).max())

max error:  0.0 0.0


In [6]:
del rx_buff, a0, a1, a2

### Filters: IIR vs overlap-and-add

In [1]:
from channel_analysis import waveform
from scipy import signal
from channel_analysis import cuda_filter
from channel_analysis.io import simulated_awgn
import cupy as cp
from cupyx.profiler import benchmark

duration = 0.1
source.sample_rate = 11.52e6  # 15.36e6
source.analysis_bandwidth = 10e6

iq = simulated_awgn(
    duration, source.sample_rate, power=source.sample_rate / source.analysis_bandwidth, xp=cp
)
cp.cuda.runtime.deviceSynchronize()

In [12]:
# filter_params = dict(
#     passband_ripple_dB=0.1,
#     stopband_attenuation_dB=90,
#     transition_bandwidth_Hz=250e3,
# )

# sos = waveform.generate_iir_lpf(
#     cutoff_Hz=source.analysis_bandwidth / 2, source.sample_rate=source.sample_rate, **filter_params
# ).astype('float32')

# out = iq.copy()

# def test_iir():
#     cuda_filter.sosfilt(sos, iq, out=out)

# sos = cp.asarray(sos)
# benchmark(test_iir, n_repeat=10, n_warmup=10)

test_iir            :    CPU:  3820.010 us   +/- 460.071 (min:  3400.992 / max:  5118.560) us     GPU-0: 40151.655 us   +/- 874.123 (min: 39627.201 / max: 42739.040) us

In [2]:
from iqwaveform import fourier
from cupyx import scipy


def test_stft(iq, Nfft=768, overlap_factor=2):
    noverlap = Nfft // overlap_factor
    freqs, times, X = fourier.spectrogram(
        iq, fs=source.sample_rate, window='hamming', nperseg=Nfft, noverlap=noverlap
    )
    q = cp.array([0.1, 0.5, 0.9, 0.99, 0.999])
    print(X.shape)

    return cp.quantile(X[:, ::2], q, axis=1)


def test_ola(iq):
    fourier.ola_filter(
        iq,
        fs=source.sample_rate,
        nperseg=768*2,
        fft_size=1024,
        window='blackman',
        passband=(-source.analysis_bandwidth / 2, source.analysis_bandwidth / 2),
    )


# ola_filter(iq)
benchmark(test_ola, (iq,), n_repeat=10, n_warmup=10)
# benchmark(test_stft, (iq,), n_repeat=10, n_warmup=10)

transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)
transform shape:  (4502, 768)


test_ola            :    CPU: 20137.015 us   +/- 1444.155 (min: 17088.435 / max: 22335.162) us     GPU-0: 30764.128 us   +/- 2199.959 (min: 26018.400 / max: 33690.975) us

In [ ]:
# X = test_stft(iq)

# def test_contig():
#     return X.view('float32')[::2]
#     return cp.ascontiguousarray(X.T).view('float32')[:,::2].T

# benchmark(test_contig, n_repeat=10, n_warmup=10)

In [ ]:
import cupy as cp

x = cp.array([1, 2, 3], dtype='complex64')
x = cp.abs(x, out=x.view('float32')[::2])